In [1]:
import matplotlib
matplotlib.use("TkAgg") #Makes imshow work on mac
from matplotlib import pyplot as plt
from Model import Model, draw_boxes
import collections
import LanguageModel
import cozmo
import pickle
import sys
import queue
import time
import threading
import numpy as np

import datetime
import pandas as pd
import csv
import base64
from ImageFeatureGen import ImageFeatureGen

Using Theano backend.


In [2]:
def detectImages():
    print('Detect Images started')
    global prediction
    
    while(True):
#         print('image queue empty? ', imageQueue.empty())
        if(not imageQueue.empty()):
#             try:
#                 print('length of image buffer? ', len(imageBuffer))
                if len(imageBuffer) < 8:
                    plt.clf() # We need to clear the plot so that we are not plotting every image each iteration. (If we don't we will get increasing delay)

                    pilImg = imageQueue.get()
                    box, imgResized = model.object_detect(pilImg.copy())
                    cleanImg = imgResized.copy();
                    draw_boxes(box, imgResized, (model.size, model.size))

                    imgWithBox = np.array(imgResized) 
                    #imgWithBox = imgWithBox[:, :, ::-1].copy()
                    box = box[0]
                    formattedBox = (box[1], box[0], box[3], box[2]) # coordinates need to be corrected for crop

                    croppedImg = cleanImg.crop(formattedBox)
                    croppedImg = np.array(croppedImg) 
                    croppedImg = croppedImg[:, :, ::-1].copy()

                    imgNumpy = np.asarray(pilImg)
                    imgStr = base64.b64encode(imgNumpy)
                    imgStr = str(imgStr)[2:-1]

                    dimensions = list(pilImg.size)[::-1] + [3]

#                     imgStr = imgStr.replace('] [', '], [')

                    row = [imgStr, prediction, dimensions, formattedBox, croppedImg]

                    if len(imageBuffer) < 8:
                        imageBuffer.append(row)

                    plt.imshow(imgWithBox)
                    plt.pause(0.001) # imshow needs time to plot the image. Need this to display the image

                else:
#                     print('predict? ', predicting)
                    if prediction is '' or predicting:
                        print('prediction? ', prediction)
                        if not imageQueue.empty():

                            predict()
#             except queue.Empty:
#                 pass

In [3]:
'''
search a list, return list of indices where the object matches the search item 
'''
def list_search(haystack: list, needle: object):
    matches = []
    i = 0
    for hay in haystack:
        if hay == needle:
            matches.append(i)
        else:
            i+=1
            continue
    return matches
            
'''
Return a list of only those objects existing in the list specified indices
'''
def slice_indices(indices: list, unsliced: list):
    sliced = []
    for i in indices:
        save = unsliced[i]
        sliced.append(save)
    return sliced

In [4]:
def predict():
    global speak
    global prediction
    
    print('Predict!')
    
    preds = []
    rows = []
    for i in range(len(imageBuffer)):
        row = imageBuffer.pop()
        currentImg = row[4]
        preds.append(predModel.predictImageWord(currentImg))
    
        rows.append(row)
        
    # each prob is a tuple ( word, probability )
    preds = [item[0] for item in preds] #remove probability number in tuple
    
    prediction = collections.Counter(preds).most_common(1)[0][0]
    
    best_guesses = list_search(preds, prediction)
    best_rows = slice_indices(best_guesses, rows)
    
    print('preds: ', preds)
    # change feedback once back to cozmo_program and user has been queried
    feedback = -1
    for r in best_rows:
        # img
        imgStr = r[0]
        dimensions = r[2]
        buffer = base64.b64decode(imgStr)
        img = np.frombuffer(buffer, dtype= np.uint8).reshape(tuple(dimensions))
        # prediction
        # box_coord
        box_coord = r[3]
        # time
        time = str(datetime.datetime.now()) #[10:]
        # feedback
        # imgFeatures
        featureGen = ImageFeatureGen()
        imgFeatures = featureGen.getFeatures(img)
        
        row = (img, prediction, box_coord, time, feedback, imgFeatures)
        exploitHistory.append(row)
    
    speak = True


   This is running predict on EVERY image in the image buffer...
    so this could be up to 8 different images, and then taking
    the median guess.
    ...
    so how to save these image features?
...
    different policies:
    --> only make a prediction if the guess is UNANIMOUS ?
    --> run down the list if made an incorrect prediction ?
    --> ask user for label; predict that label on the image; 
    DEPENDING on probability, THAT will affect how much 
    cozmo's confidence DECREASES. 
...
    OR, if that label is NOT in the dataset at all, 
    cozmo's confidence starts out at 0 for that word. 
    [overall confidence + discrete / word-level confidences]
...
    add value of prediction to the row that will be
    given to the 'exploit' dataset now that it has been called
    in the method

In [5]:
# def key_listener():
#     global predicting
#     while predicting: #True:  
# #         predicting = True
        
# #         if not imageQueue.empty():
#         print('Predict!')
# #         response = sys.stdin.readline()
# #        response = input()

#         # if confidence > threshold:
# #        if response.strip() == 'y':
#         if predicting:
#             predict()
#             predicting = False

In [6]:
def handle_image(evt, obj=None, tap_count=None,  **kwargs):
    try:
#         print('image handled')
        if(imageQueue.empty()):
            imageQueue.put_nowait(evt.image)
    except queue.Full:
        pass

In [7]:
'''
Save a history of this exploit attempt;
including:
Cozmo's prediction; 
user's feedback
'''
def exploit_history(feedback: int):
    ds = []
    for i in range(len(exploitHistory)):
        r = exploitHistory.pop()
        row = list(r)
        row[4] = feedback
        row = tuple(row)
        ds.append(row)

    datafr = pd.DataFrame(ds, columns=['img','prediction','box_coord','time','feedback','imgFeatures'])
    datafr.to_csv(exploit_dataset, index=False)
    print('save feedback')

In [8]:
def cozmo_program(robot: cozmo.robot.Robot):
    robot.set_lift_height(1.0).wait_for_completed()
    robot.camera.color_image_enabled = True
    robot.add_event_handler(cozmo.camera.EvtNewRawCameraImage, handle_image)
    print("Added event handler")
    #robot.say_text("purple").wait_for_completed()
    
    while True:
        global predicting 
        global speak
        global feedback
        if speak == True:
            print('Speak!')
            robot.say_text(prediction).wait_for_completed()
            print('??? [1 or 0]')
            feedback = input()
            exploit_history(int(feedback))
            speak = False
            predicting = True
            print('predict again? ', predicting)

        time.sleep(0.1)

In [9]:
exploit_dataset = 'exploit_history.csv'

try:
    exploit_data = pd.read_csv(exploit_dataset)
except FileNotFoundError:
    row = ['img','prediction','box_coord','time','feedback','imgFeatures']

    with open(exploit_dataset, 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(row)
        
exploit_data = pd.read_csv(exploit_dataset)

print(len(exploit_data)) 
exploit_data.head()

7


,img,prediction,box_coord,time,feedback,imgFeatures
0,[[[21 12 7]\n [21 12 7]\n [27 16 12]\n .....,green,"(35.60529, 34.4622, 219.33578, 138.15332)",2019-08-07 16:53:40.854742,0,[1.35872 0. 0. ... 2.07018 0. 2...
1,[[[21 12 7]\n [21 12 7]\n [27 16 12]\n .....,green,"(35.60529, 34.4622, 219.33578, 138.15332)",2019-08-07 16:53:32.892345,0,[1.35872 0. 0. ... 2.07018 0. 2...
2,[[[21 12 7]\n [21 12 7]\n [27 16 12]\n .....,green,"(35.60529, 34.4622, 219.33578, 138.15332)",2019-08-07 16:53:25.132958,0,[1.35872 0. 0. ... 2.07018 0. 2...
3,[[[21 12 7]\n [21 12 7]\n [27 16 12]\n .....,green,"(35.60529, 34.4622, 219.33578, 138.15332)",2019-08-07 16:53:17.346976,0,[1.35872 0. 0. ... 2.07018 0. 2...
4,[[[21 12 7]\n [21 12 7]\n [27 16 12]\n .....,green,"(35.60529, 34.4622, 219.33578, 138.15332)",2019-08-07 16:53:10.031855,0,[1.35872 0. 0. ... 2.07018 0. 2...


In [10]:
model = Model(path='../f18/data/coco2014', jpegs='../f18/train2014', bb_csv='../f18/data/coco2014/tmp/bb.csv')

with open('language-model.pickle', 'rb') as handle:
    predModel = pickle.load(handle) #trying to load a LanguageModel type

imageQueue = queue.Queue(maxsize=1)
# deque: double-ended queue; thread-safe
imageBuffer = collections.deque(maxlen=8)

exploitHistory = collections.deque(maxlen=8) 

speak = False
predicting = False
prediction = ''

/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])
/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.2 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [11]:
threading.Thread(target=detectImages).start()
# threading.Thread(target=key_listener).start()

cozmo.run_program(cozmo_program, use_viewer=True)

Detect Images started


2019-08-07 16:56:18,755 cozmo.general INFO     App connection established. sdk_version=1.4.10 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2019-08-07 16:56:18,824 cozmo.general INFO     Found robot id=1
2019-08-07 16:56:19,137 cozmo.general INFO     Connected to Android device serial=03160355293b2004
2019-08-07 16:56:19,210 cozmo.general INFO     Robot id=1 serial=024086a7 initialized OK
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/david/anaconda3/envs/oldfastai/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/home/david/anaconda3/envs/oldfastai/lib/python3.6/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/cozmo/tkview.py", line 164, in _repeat_draw_frame
    self._draw_frame()
  File "/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/cozmo/tkview.py", line 158, in _draw_frame
    self.label.con

Added event handler


/home/david/GIT/cs481-senior-design/s19/Model.py:87: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  scipy.misc.imsave(tempFile, img_resized)
/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/scipy/misc/pilutil.py:217: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  im = toimage(arr, channel_axis=2)
/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/scipy/misc/pilutil.py:381: DeprecationWarning: `bytescale` is deprecated!
`bytescale` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
  bytedata = bytescale(data, high=high, low=low, cmin=cmin, cmax=cmax)


prediction?  
Predict!
preds:  ['orange', 'orange', 'orange', 'orange', 'orange', 'green', 'orange', 'orange']


...

/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])
/home/david/GIT/cs481-senior-design/s19/Model.py:87: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  scipy.misc.imsave(tempFile, img_resized)
/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/scipy/misc/pilutil.py:217: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  im = toimage(arr, channel_axis=2)
/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages

Speak!
prediction?  orange
Predict!
??? [1 or 0]
preds:  ['orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange']
0
save feedback
predict again?  True


/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])
/home/david/GIT/cs481-senior-design/s19/Model.py:87: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  scipy.misc.imsave(tempFile, img_resized)
/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/scipy/misc/pilutil.py:217: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  im = toimage(arr, channel_axis=2)
/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages

Speak!
prediction?  orange
Predict!
??? [1 or 0]
preds:  ['green', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange']


2019-08-07 17:00:22,889 cozmo.general INFO     Shutting down connection
2019-08-07 17:00:22,890 cozmo.general INFO     Android serial=03160355293b2004 disconnected.
2019-08-07 17:00:22,891 cozmo.general INFO     Exit requested by user


### the language model that we'll be appending to w/ exploit

In [14]:
lm_df = pd.read_pickle('language-model.pickle')

In [16]:
lm_df.df.head()

AttributeError: 'LanguageModel' object has no attribute 'df'